In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
# создание сессии с помощью SparkSession

spark = SparkSession.builder.appName('Practice').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/21 12:14:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/21 12:14:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
/home/santalovdv/.local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [ ]:
# создание ссесии с помощью SparkContext

from pyspark import SparkContext, SparkConf

conf = SparkConf().setMaster("local").setAppName("newSession")

sc = SparkContext(conf=conf)

In [5]:
spark

In [ ]:
# конвертирую excel в csv

import pandas as pd

df_excel = pd.read_excel('cvm_srv.xlsx')

df_excel.to_csv('cvm_srv.csv', index=False)

In [ ]:
# чтение фрейма данных

# InferSchema - автоматическое определение типов данных при чтении файла

df_spark = spark.read.option('header', 'true').csv('cvm_srv.csv', inferSchema=True)

In [ ]:
# вывод результата

df_spark.show(1)

In [ ]:
# вывод схемы

df_spark.printSchema()

In [ ]:
# получение колонок

df_spark.columns

In [ ]:
# получение кортежа из строк
df_spark.head(1)[0]

In [ ]:
# Выборка по полю

df_spark.select('regid', 'dayy').show()

# df_spark['regid', 'dayy'].show()

In [ ]:
# Добавление поля

df_spark = df_spark.withColumn("new_column", df_spark['regid']+1)

In [ ]:
# Удаление поля

df_spark = df_spark.drop('new_column')

In [ ]:
# Переименование поля

df_spark.withColumnRenamed("regid", "regid_new")

In [ ]:
# удаление строк с пустыми полями

# drop (how = как удалять, subset = из каких полей, thresh = какая то мера удаления)

df_spark.na.drop()

In [ ]:
# чем заполнить пробелы (Null)

df_spark.na.fill('missing_values',['name_cvm'])

In [ ]:
# заполнить пробелы средним значением

# можно выбрать также mod, median

from pyspark.ml.feature import Imputer

imputer = Imputer(inputCols = ["regid","dayy","new_column"],
                 outputCols = ["{}_imputed".format(c) for c in ["regid","dayy","new_column"]]).setStrategy("mean") 

imputer.fit(df_spark).transform(df_spark).show()

In [ ]:
# фильтры

df_spark.filter("regid=68").show()

df_spark.filter(df_spark['regid']>68).show()

In [ ]:

# фильтры с условием И (&) и ИЛИ (|)

df_spark.filter((df_spark['regid']>68) & (df_spark['dayy'] == 70)).show()

df_spark.filter((df_spark['regid']>68) | (df_spark['dayy'] == 70)).show()

In [ ]:
# фильтры с условием НЕ (~)

df_spark.filter(~(df_spark['regid']>68)).show()

In [6]:
# создание DataFrame (самостоятельно)

data = [('Lucy', 10, 3_000),('Tanya', 35, 200_000), ('Kolya', 15, 0)]

df = spark.createDataFrame(data, ['name', 'age', 'money'])
